In [1]:
import numpy as np
import pandas as pd
import psycopg2

### Connect to Postgres DB

In [2]:
import json

with open('config.json') as f:
    conf = json.load(f)
    host = conf['host']
    database = conf['database']
    user = conf['user']
    passw = conf['passw']

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)
conn = psycopg2.connect(conn_str)

### Initial Query

In [4]:
Vic_Off_query = '''SELECT vic.victim_id AS VICTIM_ID,
            vic.incident_id AS INCIDENT_ID,
            date_part('year',inc.incident_date),
            vic.victim_type_id AS VICTIM_TYPE_ID,
            ty.victim_type_name AS VICTIM_TYPE,
            vic.age_range_low_num AS AGE_RANGE_LOW,
            vic.age_range_high_num AS AGE_RANGE_HIGH,
            vic.sex_code AS VICTIM_SEX,
            oft.crime_against AS CRIME_AGAINST,
            oft.offense_name AS OFFENSE,
            oft.offense_category_name AS OFFENSE_CATEGORY,
            oft.offense_group AS OFFENSE_GROUP,
            ofr.offender_id AS OFFENDER_ID,
            ofr.age_num AS OFFENDER_AGE,
            ofr.sex_code AS OFFENDER_SEX,
            off.location_id AS LOCATION_ID,
            loc.location_name AS LOCATION_NAME,
            inc.ddocname AS DOC_NAME,
            ori.fips AS FIPS,
            ori.countyname AS COUNTY,
            ori.name AS ORI_NAME,
            ags.population AS POPULATION,
            ags.population_group_desc AS POPULATION_DESCRIPTION,
            ags.total_officers AS OFFICERS,
            ags.total_civilians AS CIVILIANS
            FROM nibrs_victim as vic
            JOIN nibrs_victim_type as ty
            ON vic.victim_type_id = ty.victim_type_id
            JOIN nibrs_offender as ofr
            ON ofr.incident_id = vic.incident_id
            JOIN nibrs_offense as off
            ON off.incident_id = vic.incident_id
            JOIN nibrs_offense_type as oft
            ON oft.offense_type_id = off.offense_type_id
            JOIN nibrs_location_type as loc
            ON off.location_id = loc.location_id
            JOIN nibrs_incident as inc
            ON inc.incident_id = vic.incident_id
            JOIN cde_agencies as ags
            ON ags.agency_id = inc.agency_id
            JOIN ori_to_fips as ori
            ON ori.ori9 = ags.ori
            WHERE vic.victim_type_id = 4;'''

### Load into Pandas DataFrame

In [ ]:
Vic_Off_df = pd.read_sql(Vic_Off_query, con=conn)
Vic_Off_df.head(3).T

In [7]:
Vic_Off_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463095 entries, 0 to 463094
Data columns (total 12 columns):
incident_id         463095 non-null int64
year                463095 non-null int64
age_num             456081 non-null float64
victim_sex          463095 non-null object
crime_against       463095 non-null object
offense             463095 non-null object
offense_category    463095 non-null object
location_id         463095 non-null int64
location_name       463095 non-null object
county              463095 non-null object
officers            463095 non-null int64
civilians           463095 non-null int64
dtypes: float64(1), int64(5), object(6)
memory usage: 42.4+ MB


In [8]:
Vic_Off_df = Vic_Off_df.drop_duplicates()

In [9]:
Vic_Off_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463095 entries, 0 to 463094
Data columns (total 12 columns):
incident_id         463095 non-null int64
year                463095 non-null int64
age_num             456081 non-null float64
victim_sex          463095 non-null object
crime_against       463095 non-null object
offense             463095 non-null object
offense_category    463095 non-null object
location_id         463095 non-null int64
location_name       463095 non-null object
county              463095 non-null object
officers            463095 non-null int64
civilians           463095 non-null int64
dtypes: float64(1), int64(5), object(6)
memory usage: 45.9+ MB


In [10]:
Vic_Off_df.head(3).T

,0,1,2
incident_id,67693343,67693344,67693345
year,2013,2013,2013
age_num,46,57,51
victim_sex,F,M,F
crime_against,Property,Property,Property
offense,All Other Larceny,Theft From Motor Vehicle,Theft From Motor Vehicle
offense_category,Larceny/Theft Offenses,Larceny/Theft Offenses,Larceny/Theft Offenses
location_id,20,18,18
location_name,Residence/Home,Parking/Drop Lot/Garage,Parking/Drop Lot/Garage
county,POTTER,POTTER,POTTER


### Count of NaN's by column

In [11]:
Vic_Off_df.isna().sum()

incident_id            0
year                   0
age_num             7014
victim_sex             0
crime_against          0
offense                0
offense_category       0
location_id            0
location_name          0
county                 0
officers               0
civilians              0
dtype: int64

### Impute missing offender_sex as "U" (unknown)

In [ ]:
Vic_Off_df.offender_sex.value_counts()

In [ ]:
Vic_Off_df['offender_sex'].fillna('U', inplace=True)

In [ ]:
Vic_Off_df.offender_sex.value_counts()

# Plot number of offenses by offender age

In [ ]:
OffAges = Vic_Off_df['offender_age'].value_counts().sort_index()
OffAges

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

OffAges.plot.bar(figsize = (20,20))
plt.title('Number of Offenses by Offender Age')
plt.xlabel('Age')
plt.ylabel('Offense Count')
#plt.rcParams.update({'font.size': 40})
plt.show()

In [ ]:
import seaborn as sb

In [ ]:
sb.distplot(OffAges)
plt.title('Number of Offenses by Offender Age')
plt.xlabel('Age')
plt.ylabel('Offense Count')
#plt.rcParams.update({'font.size': 40})
plt.show()

# Plot Victim Count by Gender and County

In [ ]:
small_df = Vic_Off_df[['victim_sex','fips']]

In [ ]:
small_df['fips'].unique()

In [ ]:
small_df.columns = ['victim_sex','fips']

axarr = small_df.hist(column='fips', by = 'victim_sex', sharex=True, sharey=True, layout = (1, 3), figsize=(15,5))

for ax in axarr.flatten():
    ax.set_xlabel("FIPS")
    ax.set_ylabel("Victim Count")

# Plot total number of victims by FIPS

In [ ]:
FIPS_Vics = pd.value_counts(small_df['fips'].values, sort=True)
FIPS_Vics

In [ ]:
FIPS_Vics.plot.barh(figsize = (20,30))
plt.title('Number of Victims by FIPS')
plt.xlabel('FIPS')
plt.ylabel('Victim Count')
plt.rcParams.update({'font.size': 16})
plt.show()

### Plot Number of Offenses by Category

In [ ]:
OffCats = Vic_Off_df['offense_category'].value_counts().sort_index()
OffCats

In [ ]:
OffCats.plot.barh(figsize = (20,20))
plt.title('Number of Incidents by Offense Category')
plt.xlabel('Offense Category')
plt.ylabel('Incident Count')
plt.rcParams.update({'font.size': 26})
plt.show()